In [1]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import ConvexHull
import folium

In [2]:
data = pd.read_csv('../data/data_combined_v1.04.csv')\
    [['place_id',
    'name',
    'price_level_combined',
    'user_ratings_total',
    'combined_rating',
    'geometry.location.lat', 'geometry.location.lng',
    'combined_main_category',
    'sub_category',
    'district', 
    'district_cluster',
    'combined_main_category_2']]
data

,place_id,name,price_level_combined,user_ratings_total,combined_rating,geometry.location.lat,geometry.location.lng,combined_main_category,sub_category,district,district_cluster,combined_main_category_2
0,ChIJd6kh67ZkjEcRC1IQourQOyo,Restaurant La Tentation,3.0,158.0,4.1,46.205232,6.114945,"European, Swiss",NaN,Saint-Jean Charmilles,3,European
1,ChIJX68uubZkjEcRKKxpZ7OrgqU,Café Restaurant Palma,4.0,141.0,3.7,46.204931,6.114021,General / Restaurant,NaN,Saint-Jean Charmilles,3,General
2,ChIJPcYsG_BljEcR4kTFqBJJoK4,Le Versailles,3.0,56.0,3.9,46.205752,6.114763,"Italian, French, European",NaN,Saint-Jean Charmilles,3,European
3,ChIJV8uySFxljEcRn7XpIO3zZX0,Restaurant Les Jardins du Rhône,NaN,NaN,NaN,46.205248,6.115503,General / Restaurant,NaN,Saint-Jean Charmilles,3,General
4,ChIJW8zU8rZkjEcR_Ke8t7c7hVc,Canteen De La Residence Gardens Du Rhône,4.0,1.0,5.0,46.205387,6.115347,General / Restaurant,NaN,Saint-Jean Charmilles,3,General
...,...,...,...,...,...,...,...,...,...,...,...,...
1563,ChIJtW8VekZljEcRHMTm4W0SQoo,Hôtel du Parc des Eaux-Vives,4.0,477.0,4.4,46.207746,6.169275,General / Restaurant,NaN,Eaux-Vives - Lac,14,General
1564,ChIJ8xcfO5JljEcRMsNfQ7YjlqM,La Terrasse du Parc,4.0,10.0,3.0,46.208511,6.169342,European,NaN,Eaux-Vives - Lac,14,European
1565,ChIJhxG2srN6jEcRUls_UY7b5Io,Le B d'Armand,3.0,100.0,4.0,46.192811,6.169873,General / Fast food / Snacks / Take Away,Sandwich / Snacks,Champel,36,Fast Food
1566,ChIJHSzAC1JljEcRjr7q009gZe4,Road Runner,3.0,376.0,4.1,46.199892,6.174117,American,NaN,Eaux-Vives - Lac,14,American


In [3]:
# coordinates for district clusters
df_cluster_centers = pd.read_csv('../data/data_cluster_centers_v1.02.csv').rename(columns={'district_area':'district_cluster'})
df_cluster_centers

,district_cluster,cluster_center_lat,cluster_center_lng
0,1,46.206097,6.128191
1,2,46.212504,6.129205
2,3,46.208559,6.120886
3,4,46.222885,6.127898
4,5,46.219508,6.137469
5,6,46.214862,6.131804
6,7,46.202012,6.137967
7,8,46.200854,6.130934
8,9,46.199478,6.134993
9,10,46.196314,6.139261


Main Page

- There xx is rest in GVA
    - Most common xx rest, followed by yy
    - Least common one xx rest, followed by yy

- Most restaurants are in aa district followed by bb
- Least restaurants are in cc followed by

When you select 
District Page
   - There xx is rest in GVA
   - Most common xx rest, followed by yy
   - Least common one xx rest, followed by yy
    
   - Most densed area xx
   - Least densed area yy
   
Should be similar when you select restaurant

# Functions

In [4]:
def filter_data(data, rest_district, rest_category_main, rest_category):
    """
    Filters main dataframe based on district or restaurant selection
    FOR DROPDOWN MENUS
    Returns a filtered dataframe
    """
    if rest_district != 'All':
        data = data[data['district']==rest_district]
        
    if rest_category_main != 'All':
        data = data[data['combined_main_category_2']==rest_category_main]
        
    if rest_category != 'All':
        data = data[data['combined_main_category'].str.contains(rest_category)]
        
    return data.reset_index(drop=True)

In [5]:
filter_data(data, 'All', 'European', "Italian")

,place_id,name,price_level_combined,user_ratings_total,combined_rating,geometry.location.lat,geometry.location.lng,combined_main_category,sub_category,district,district_cluster,combined_main_category_2
0,ChIJPcYsG_BljEcR4kTFqBJJoK4,Le Versailles,3.0,56.0,3.9,46.205752,6.114763,"Italian, French, European",NaN,Saint-Jean Charmilles,3,European
1,ChIJC_DLKcdkjEcRexbawoQqn0g,Auberge Fédérale - Le Ginevra,3.0,313.0,4.2,46.212803,6.123381,"Italian, Swiss, European",NaN,Saint-Jean Charmilles,3,European
2,ChIJdRFop8hkjEcRgxjZITIYzBs,Restaurant Francesca Genève,4.0,10.0,4.5,46.208549,6.124254,Italian,NaN,Saint-Jean Charmilles,3,European
3,ChIJIyvjXKtljEcR_8tG92ZUEGY,LE FARINET Restaurant Italien,4.0,63.0,4.3,46.204245,6.124537,Italian,NaN,Saint-Jean Charmilles,1,European
4,ChIJ4Yr9FlFljEcR1R8FSCAWg94,"Brasserie Nouvelle, Restaurant Italien sur Genève",3.0,160.0,3.9,46.208731,6.125641,"Italian, European, French",NaN,Saint-Jean Charmilles,1,European
...,...,...,...,...,...,...,...,...,...,...,...,...
101,ChIJTeU4l2BljEcR9VNtylfbfnY,Ragù Ristorante Trattoria,4.0,79.0,4.8,46.201104,6.160913,Italian,NaN,Eaux-Vives - Lac,17,European
102,ChIJmV19Wl1ljEcRQwFXEfeG-RU,Scaramu Restaurant Pizzeria,3.0,171.0,4.2,46.200312,6.163688,"Italian, European",NaN,Eaux-Vives - Lac,17,European
103,ChIJ10TcxEtljEcR-10XIINiAE8,Café Restaurant des Voyageurs,3.0,241.0,4.5,46.200462,6.166393,"French, Swiss, Italian",NaN,Eaux-Vives - Lac,14,European
104,ChIJL0ES1hNljEcRt8nDWJ1hxuk,Gigi Cucina & Bar,3.0,290.0,4.3,46.200999,6.166493,Italian,NaN,Eaux-Vives - Lac,14,European


In [6]:
def filter_data_scoring(data, rest_district, rest_category_main, rest_category):
    """
    Filters main dataframe based on district or restaurant selection
    FOR SCORING
    Returns a filtered dataframe
    """
    if rest_district != 'All':
        data = data[data['district']==rest_district]
        
    if rest_category_main != 'All':
        data = data[data['combined_main_category_2']==rest_category_main]
        
    if rest_category != 'All':
        data = data[data['combined_main_category'].str.contains(rest_category)]

    data = data.groupby(['district','district_cluster'])\
        [['place_id', 'user_ratings_total','combined_rating']]\
        .agg({'place_id':'count',
        'user_ratings_total':'mean',
        'combined_rating':'mean'})\
        .rename(columns={'place_id':f'{rest_category.lower()}_restaurants'})
        
    return data.reset_index()

In [7]:
filter_data_scoring(data, 'All', 'European', "Italian")

,district,district_cluster,italian_restaurants,user_ratings_total,combined_rating
0,Bâtie - Acacias,11,1,88.000000,4.000000
1,Bâtie - Acacias,12,1,210.000000,4.100000
2,Bâtie - Acacias,13,1,201.000000,4.400000
3,Champel,37,3,137.000000,3.600000
4,Cité-Centre,32,3,249.000000,4.300000
5,Cité-Centre,33,3,156.000000,4.250000
6,Cité-Centre,34,6,164.333333,4.000000
7,Cité-Centre,35,12,392.000000,4.233333
8,Eaux-Vives - Lac,14,3,255.333333,4.400000
9,Eaux-Vives - Lac,15,8,255.571429,4.462500


In [8]:
data.columns

Index(['place_id', 'name', 'price_level_combined', 'user_ratings_total',
       'combined_rating', 'geometry.location.lat', 'geometry.location.lng',
       'combined_main_category', 'sub_category', 'district',
       'district_cluster', 'combined_main_category_2'],
      dtype='object')

In [9]:
def merge_data(data, rest_district, rest_category_main, rest_category):
    """
    Creates a merged data set based on filtering selections and
    returns the final data set before normalization and scoring    
    """
    if rest_category == 'All':
        data = filter_data_scoring(data, rest_district, rest_category_main, rest_category)
    else:
        data = filter_data_scoring(data, rest_district, 'All', 'All')\
            .merge(
                filter_data_scoring(data, rest_district, rest_category_main, rest_category)\
                    .drop(columns=['district','user_ratings_total','combined_rating']),
                how='left',
                on='district_cluster')\
            .fillna(0)   
    return data

In [10]:
# Testing all conditions
rest_district = 'Bâtie - Acacias'
rest_category = 'Italian'
rest_category_main = 'European'
df = merge_data(data, rest_district, rest_category_main, rest_category)
df

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,italian_restaurants
0,Bâtie - Acacias,11,19,145.470588,4.347059,1
1,Bâtie - Acacias,12,15,306.384615,4.261538,1
2,Bâtie - Acacias,13,14,95.461538,4.330769,1


In [11]:
def score_data(data, rest_district, rest_category_main, rest_category, score_com, score_pop, score_sat):
    """
    Normalizes merged data set and create a custom scoring
    """
    # create merged data set
    df_merged = merge_data(data, rest_district, rest_category_main, rest_category)
    
    # normalization
    scaler = MinMaxScaler()
    cols = df_merged.drop(columns=['district','district_cluster'])
    scaler.fit(cols)
    df_score = pd.DataFrame(scaler.transform(cols), columns=cols.columns+'_norm')
    
    # scoring
    df_score['score'] = (1-df_score['all_restaurants_norm'])\
                        + df_score['user_ratings_total_norm']\
                        + (1-df_score['combined_rating_norm'])

    if rest_category != 'All':
        df_score['score'] += (1-df_score[f'{rest_category.lower()}_restaurants_norm'])
    
    # create output data_set
    df_output = pd.concat([df_merged, df_score], axis=1)\
        .merge(df_cluster_centers, how='left', on='district_cluster')
    return df_output

In [12]:
def score_data(data, rest_district, rest_category_main, rest_category, score_com, score_pop, score_sat):
    """
    Normalizes merged data set and create a custom scoring
    """
    # create merged data set
    df_merged = merge_data(data, rest_district, rest_category_main, rest_category)
    
    # normalization
    scaler = MinMaxScaler()
    cols = df_merged.drop(columns=['district','district_cluster'])
    scaler.fit(cols)
    df_score = pd.DataFrame(scaler.transform(cols), columns=cols.columns+'_norm')
    
    # scoring
    
    if rest_category == 'All':
        score_tot = score_com + score_pop + score_sat
        df_score['score'] = (score_com * (1-df_score['all_restaurants_norm'])\
                            +score_pop * df_score['user_ratings_total_norm']\
                            +score_sat * (1-df_score['combined_rating_norm']))\
                            / score_tot

    else:
        score_tot = 2 * score_com + score_pop + score_sat
        df_score['score'] = (score_com * (1-df_score['all_restaurants_norm'])\
                            +score_pop * df_score['user_ratings_total_norm']\
                            +score_sat * (1-df_score['combined_rating_norm'])
                            +score_com * (1-df_score[f'{rest_category.lower()}_restaurants_norm']))\
                            / score_tot
    
    # create output data_set
    df_output = pd.concat([df_merged, df_score], axis=1)\
        .merge(df_cluster_centers, how='left', on='district_cluster')
    return df_output

In [13]:
score_data(data, 'Cité-Centre', 'Asian', 'Japanese', 1, 4, 1).sort_values(by='score', ascending=False)

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,japanese_restaurants,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,japanese_restaurants_norm,score,cluster_center_lat,cluster_center_lng
0,Cité-Centre,32,89,336.147059,4.118841,8,1.000000,1.000000,0.228648,1.0,0.681622,46.203232,6.145410
3,Cité-Centre,35,86,307.507463,4.094203,5,0.928571,0.698504,0.022684,0.5,0.620394,46.202112,6.149501
1,Cité-Centre,33,47,290.022727,4.211111,2,0.000000,0.514437,1.000000,0.0,0.579678,46.201271,6.154714
2,Cité-Centre,34,55,241.155556,4.091489,5,0.190476,0.000000,0.000000,0.5,0.329932,46.201183,6.141425


In [14]:
score_data(data, 'All', 'Asian', 'Japanese', 1, 4, 1).sort_values(by='score', ascending=False)

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,japanese_restaurants,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,japanese_restaurants_norm,score,cluster_center_lat,cluster_center_lng
34,Servette Petit-Saconnex,4,17,377.750000,4.375000,1.0,0.046512,1.000000,0.801257,0.125,0.861033,46.222885,6.127898
1,Bâtie - Acacias,12,15,306.384615,4.261538,0.0,0.023256,0.747190,0.522867,0.000,0.777520,46.190460,6.130733
6,Cité-Centre,33,47,290.022727,4.211111,2.0,0.395349,0.689228,0.399138,0.250,0.673204,46.201271,6.154714
14,Grottes Saint-Gervais,19,74,318.539683,4.048438,8.0,0.709302,0.790249,0.000000,1.000,0.635956,46.209640,6.143661
8,Cité-Centre,35,86,307.507463,4.094203,5.0,0.848837,0.751168,0.112291,0.625,0.631220,46.202112,6.149501
5,Cité-Centre,32,89,336.147059,4.118841,8.0,0.883721,0.852623,0.172742,1.000,0.622004,46.203232,6.145410
31,Saint-Jean Charmilles,1,30,215.034483,4.244828,1.0,0.197674,0.423584,0.481865,0.125,0.555685,46.206097,6.128191
7,Cité-Centre,34,55,241.155556,4.091489,5.0,0.488372,0.516118,0.105633,0.625,0.549352,46.201183,6.141425
26,La Cluse - Philosophes,31,34,225.406250,4.334375,2.0,0.244186,0.460326,0.701580,0.250,0.520791,46.196324,6.143047
29,Pâquis Sécheron,26,77,223.969697,4.220290,2.0,0.744186,0.455237,0.421659,0.250,0.486443,46.210767,6.149114


In [15]:
def pick_location(data, rest_district, rest_category_main, rest_category, score_com, score_pop, score_sat):
    """    
    Select best / worst location based on custom scoring
    """
    if rest_district == 'All':
        n = 10
    else:
        n = 1
        
    best_location = score_data(data, rest_district, rest_category_main, rest_category, score_com, score_pop, score_sat)\
        .nlargest(n, 'score').reset_index(drop=True)
    worst_location = score_data(data, rest_district, rest_category_main, rest_category, score_com, score_pop, score_sat)\
        .nsmallest(n, 'score').reset_index(drop=True)

    return best_location, worst_location

In [16]:
# best locations
pick_location(data, 'All', 'American', 'Mexican', 5, 1, 1)[0]

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,mexican_restaurants,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,mexican_restaurants_norm,score,cluster_center_lat,cluster_center_lng
0,Bâtie - Acacias,12,15,306.384615,4.261538,0.0,0.023256,0.747190,0.522867,0.0,0.925670,46.190460,6.130733
1,Servette Petit-Saconnex,4,17,377.750000,4.375000,0.0,0.046512,1.000000,0.801257,0.0,0.913849,46.222885,6.127898
2,Servette Petit-Saconnex,5,13,129.272727,4.218182,0.0,0.000000,0.119775,0.416487,0.0,0.891941,46.219508,6.137469
3,Pâquis Sécheron,27,23,121.857143,4.136364,0.0,0.116279,0.093506,0.215737,0.0,0.858031,46.222153,6.145509
4,Bâtie - Acacias,13,14,95.461538,4.330769,0.0,0.011628,0.000000,0.692732,0.0,0.854094,46.190399,6.136831
5,Champel,36,24,154.684211,4.189474,0.0,0.127907,0.209795,0.346048,0.0,0.852018,46.196470,6.161431
6,Eaux-Vives - Lac,14,19,164.266667,4.323529,0.0,0.069767,0.243740,0.674969,0.0,0.851661,46.201911,6.167926
7,Bâtie - Acacias,11,19,145.470588,4.347059,0.0,0.069767,0.177156,0.732701,0.0,0.841301,46.191380,6.138648
8,Saint-Jean Charmilles,1,30,215.034483,4.244828,0.0,0.197674,0.423584,0.481865,0.0,0.829446,46.206097,6.128191
9,Saint-Jean Charmilles,3,33,180.269231,4.246154,0.0,0.232558,0.300429,0.485119,0.0,0.804377,46.208559,6.120886


In [17]:
# best locations
pick_location(data, 'All', 'American', 'Mexican', 1, 5, 5)[0]

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,mexican_restaurants,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,mexican_restaurants_norm,score,cluster_center_lat,cluster_center_lng
0,Grottes Saint-Gervais,19,74,318.539683,4.048438,2.0,0.709302,0.790249,0.000000,0.666667,0.797940,46.209640,6.143661
1,Cité-Centre,32,89,336.147059,4.118841,0.0,0.883721,0.852623,0.172742,0.000000,0.792974,46.203232,6.145410
2,Cité-Centre,35,86,307.507463,4.094203,0.0,0.848837,0.751168,0.112291,0.000000,0.778796,46.202112,6.149501
3,Cité-Centre,34,55,241.155556,4.091489,1.0,0.488372,0.516118,0.105633,0.333333,0.685893,46.201183,6.141425
4,Bâtie - Acacias,12,15,306.384615,4.261538,0.0,0.023256,0.747190,0.522867,0.000000,0.674863,46.190460,6.130733
5,Cité-Centre,33,47,290.022727,4.211111,0.0,0.395349,0.689228,0.399138,0.000000,0.671259,46.201271,6.154714
6,Servette Petit-Saconnex,4,17,377.750000,4.375000,0.0,0.046512,1.000000,0.801257,0.000000,0.662267,46.222885,6.127898
7,Saint-Jean Charmilles,1,30,215.034483,4.244828,0.0,0.197674,0.423584,0.481865,0.000000,0.542577,46.206097,6.128191
8,Pâquis Sécheron,26,77,223.969697,4.220290,0.0,0.744186,0.455237,0.421659,0.000000,0.535309,46.210767,6.149114
9,Pâquis Sécheron,27,23,121.857143,4.136364,0.0,0.116279,0.093506,0.215737,0.000000,0.522714,46.222153,6.145509


In [18]:
# best locations
pick_location(data, 'All', 'All', 'All', 1, 4, 1)[0]

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,score,cluster_center_lat,cluster_center_lng
0,Servette Petit-Saconnex,4,17,377.750000,4.375000,0.046512,1.000000,0.801257,0.858705,46.222885,6.127898
1,Grottes Saint-Gervais,19,74,318.539683,4.048438,0.709302,0.790249,0.000000,0.741949,46.209640,6.143661
2,Bâtie - Acacias,12,15,306.384615,4.261538,0.023256,0.747190,0.522867,0.740439,46.190460,6.130733
3,Cité-Centre,32,89,336.147059,4.118841,0.883721,0.852623,0.172742,0.725671,46.203232,6.145410
4,Cité-Centre,35,86,307.507463,4.094203,0.848837,0.751168,0.112291,0.673924,46.202112,6.149501
5,Cité-Centre,33,47,290.022727,4.211111,0.395349,0.689228,0.399138,0.660404,46.201271,6.154714
6,Cité-Centre,34,55,241.155556,4.091489,0.488372,0.516118,0.105633,0.578411,46.201183,6.141425
7,Saint-Jean Charmilles,1,30,215.034483,4.244828,0.197674,0.423584,0.481865,0.502466,46.206097,6.128191
8,La Cluse - Philosophes,31,34,225.406250,4.334375,0.244186,0.460326,0.701580,0.482590,46.196324,6.143047
9,Jonction - Plainpalais,10,44,229.682927,4.368293,0.360465,0.475476,0.784800,0.459440,46.196314,6.139261


In [20]:
# worst locations
pick_location(data, 'All', 'All', 'All', 5, 1, 1)[1]

,district,district_cluster,all_restaurants,user_ratings_total,combined_rating,all_restaurants_norm,user_ratings_total_norm,combined_rating_norm,score,cluster_center_lat,cluster_center_lng
0,Pâquis Sécheron,24,99,235.662791,4.273864,1.000000,0.496660,0.553108,0.134793,46.211422,6.145627
1,Cité-Centre,32,89,336.147059,4.118841,0.883721,0.852623,0.172742,0.323039,46.203232,6.145410
2,Eaux-Vives - Lac,15,69,153.709677,4.309375,0.651163,0.206343,0.640239,0.330041,46.203589,6.157793
3,Pâquis Sécheron,26,77,223.969697,4.220290,0.744186,0.455237,0.421659,0.330378,46.210767,6.149114
4,Cité-Centre,35,86,307.507463,4.094203,0.848837,0.751168,0.112291,0.342099,46.202112,6.149501
5,Grottes Saint-Gervais,21,64,165.333333,4.361818,0.593023,0.247519,0.768914,0.359070,46.206649,6.141928
6,Pâquis Sécheron,25,61,153.288462,4.388679,0.558140,0.204850,0.834821,0.368476,46.214645,6.148139
7,Jonction - Plainpalais,9,51,99.195652,4.450000,0.441860,0.013228,0.985278,0.402664,46.199478,6.134993
8,Jonction - Plainpalais,8,52,106.244444,4.360000,0.453488,0.038198,0.764453,0.429472,46.200854,6.130934
9,Grottes Saint-Gervais,19,74,318.539683,4.048438,0.709302,0.790249,0.000000,0.463391,46.209640,6.143661


# Test & Visualize

In [ ]:
import plotly.express as px
px.set_mapbox_access_token(open("../.env/.mapbox_access_token").read())

In [ ]:
rest_district = 'Jonction - Plainpalais'
rest_category = 'Japanese'
rest_category_main = 'Asian'

fig = px.scatter_mapbox(pick_location(data, rest_district, rest_category_main, rest_category)[0], 
                        lat="cluster_center_lat", 
                        lon="cluster_center_lng", 
                        color="district_cluster",
                        opacity=1,
                        center={'lat':46.2043907,'lon':6.1431577},
                        width=800,
                        height=600,
                        zoom=12)
fig.show()

In [ ]:
rest_district = 'All'
rest_category = 'All'
rest_category_main = 'All'
score_data(data, rest_district, rest_category_main, rest_category, 0, 4, 0).sort_values(by='score', ascending=False)

In [ ]:
data[(data['district']=='Servette Petit-Saconnex')]['user_ratings_total'].mean()

In [ ]:
data[data['district']=='Servette Petit-Saconnex']['user_ratings_total'].median()

In [ ]:
data.groupby(['district', 'district_cluster'])['user_ratings_total'].agg(['mean', 'median'])

In [ ]:
rest_district = 'Eaux-Vives - Lac'
rest_category = 'Asian'
rest_category_main = 'Thai'
score_data(data, rest_district, rest_category_main, rest_category).nlargest(10, 'score')

In [ ]:
data[data['combined_main_category_2']=='Asian']

In [ ]:
data[data['district']=='Servette Petit-Saconnex']

In [ ]:
data['price_level_combined'].value_counts()

In [30]:
def create_convexhull_polygon(map_object, list_of_points, layer_name, line_color, fill_color, weight, text): 

    # Since it is pointless to draw a convex hull polygon around less than 3 points check len of input
    if len(list_of_points) > 2:

        # Create the convex hull using scipy.spatial 
        form = [list_of_points[i] for i in ConvexHull(list_of_points).vertices]

        # Create feature group, add the polygon and add the feature group to the map 
        fg = folium.FeatureGroup(name=layer_name)
        fg.add_child(folium.vector_layers.Polygon(locations=form, color=line_color, fill_color=fill_color,stroke=False,
                                                  weight=weight, popup=(folium.Popup(text))))
        map_object.add_child(fg)

    return (map_object)

In [31]:
best_locations = pick_location(data, 'All', 'All', 'All', 2, 2, 2)[0]
worst_locations = pick_location(data, 'All', 'All', 'All', 2, 2, 2)[1]


my_convexhull_map = folium.Map(location=[46.20494053262858, 6.142254182958967], zoom_start=13, max_zoom=14, min_zoom=12, tiles='cartodbpositron')

for i, row in best_locations.iterrows():
    cluster = row['district_cluster']
    list_of_points = data[data['district_cluster']==cluster][['geometry.location.lat','geometry.location.lng']].to_numpy()
    create_convexhull_polygon(my_convexhull_map, list_of_points, layer_name='Best Locations',
                        line_color='green',
                        fill_color='green',
                        weight=1,
                        text=f"{row['district_cluster']}")
my_convexhull_map

In [32]:
total_map = score_data(data, 'All', 'All', 'All', 2, 2, 2).sort_values(by='score', ascending=False)

my_convexhull_map = folium.Map(location=[46.20494053262858, 6.142254182958967], zoom_start=13.4, max_zoom=14, min_zoom=12, tiles='cartodbpositron')

for i, row in total_map.iterrows():
    cluster = row['district_cluster']
    list_of_points = data[data['district_cluster']==cluster][['geometry.location.lat','geometry.location.lng']].to_numpy()
    create_convexhull_polygon(my_convexhull_map, list_of_points, layer_name='Best Locations',
                        line_color='blue',
                        fill_color='blue',
                        weight=1,
                        text=f"{row['district_cluster']}")
my_convexhull_map

In [ ]:
lat = df_district[df_district['district']==rest_district]['district_lat'][0]
lat

In [ ]:
lng = df_district[df_district['district']==rest_district]['district_lng']
lng

In [ ]:
lng = df_district[df_district['district']==rest_district]['district_zoom']
lng

In [ ]:
df_district[df_district['district']==rest_district][['district_zoom']][0]